# DataFrameMethodTransformer
This notebook shows the functionality in the `DataFrameMethodTransformer` class. This transformer applys a `pd.DataFrame` method to the input `X`. <br>
This generic transformer means that many `pd.DataFrame` methods are available for use within the package without having to directly implement a transformer for that specific function.

In [1]:
import pandas as pd
import numpy as np

In [2]:
import tubular
from tubular.base import DataFrameMethodTransformer

In [3]:
tubular.__version__

'0.2.8'

## Load Boston house price dataset from sklearn
Note, the load_boston script modifies the original Boston dataset to include nulls values and pandas categorical dtypes.

In [4]:
boston_df = tubular.testing.test_data.prepare_boston_df()

In [5]:
boston_df.shape

(506, 17)

In [6]:
boston_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target,ZN_cat,CHAS_cat,RAD_cat
0,0.00632,18.0,2.31,0.0,0.538,6.575,NaN,4.0900,NaN,296.0,15.3,396.90,4.98,24.0,18.0,0.0,NaN
1,0.02731,NaN,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,NaN,0.0,2.0
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,NaN,17.8,392.83,4.03,34.7,0.0,0.0,2.0
3,NaN,NaN,2.18,0.0,0.458,NaN,45.8,6.0622,3.0,222.0,18.7,NaN,NaN,33.4,NaN,0.0,3.0
4,0.06905,0.0,2.18,0.0,0.458,NaN,NaN,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,0.0,0.0,3.0


## Simple usage

### Initialising DataFrameMethodTransformer

The user must specify the following; <br>
- `new_column_name` the name or names of columns to assign the outputs of the `pd.DataFrame` method to <br> 
- `pd_method_name` the name of the `pd.DataFrame` method to be called <br>
- `columns` the columns in the `DataFrame` passed to the `transform` method to be transformed <br>
- `pd_method_kwargs` a dictionary of keyword arguments that are passed to the `pd.DataFrame` method when called <br>

Note, for `DataFrameMethodTransformer` the `columns` argument is mandatory. This is different to most of the other transformers in the package, which will pick up all columns in the data to use if it is not supplied. The reason for this is that is it very unlikely a user will want to run this transformer on all columns.

In [7]:
sum_transformer = DataFrameMethodTransformer(
    columns = ['CRIM', 'INDUS'], 
    pd_method_name = 'sum',
    new_column_name = 'CRIM_INDUS_sum', 
    pd_method_kwargs = {'axis': 1}
)

### DataFrameMethodTransformer fit
There is no fit method for the DataFrameMethodTransformer as the methods that it can run do not 'learn' anything from the data.

### DataFrameMethodTransformer transform
When running transform with this configuration a new column `CRIM_INDUS_sum` is added to the input `X` which is the sum of `CRIM` and `INDUS`.

In [8]:
boston_df_2 = sum_transformer.transform(boston_df)

In [9]:
boston_df_2[['CRIM', 'INDUS', 'CRIM_INDUS_sum']].head()

,CRIM,INDUS,CRIM_INDUS_sum
0,0.00632,2.31,2.31632
1,0.02731,7.07,7.09731
2,0.02729,7.07,7.09729
3,NaN,2.18,2.18000
4,0.06905,2.18,2.24905


## Multiple column assignment

It is possible to assign the output of the `pd.DataFrame` method to multiple columns by passing a list of column names to `new_column_name`.

In [10]:
div_transformer = DataFrameMethodTransformer(
    columns = ['CRIM', 'INDUS'], 
    pd_method_name = 'div',
    new_column_name = ['CRIM_half', 'INDUS_half'], 
    pd_method_kwargs = {'other': 2}
)

In [11]:
boston_df_3 = div_transformer.transform(boston_df)

In [12]:
boston_df_3[['CRIM', 'INDUS', 'CRIM_half', 'INDUS_half']].head()

,CRIM,INDUS,CRIM_half,INDUS_half
0,0.00632,2.31,0.003160,1.155
1,0.02731,7.07,0.013655,3.535
2,0.02729,7.07,0.013645,3.535
3,NaN,2.18,NaN,1.090
4,0.06905,2.18,0.034525,1.090


## Other examples 

Below are other examples of using the `DataFrameMethodTransformer` transformer with exisitng `pd.DataFrame` numerical methods. <br> 
It is possible to use any [pd.DataFrame method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.abs.html), although some may not work correctly. The transformer only checks that the supplied method is available from the `pd.DataFrame` class. There are many combinations of # output columns, input columns, method and method keyword args that it cannot check.

### Cumulative sum

In [13]:
cumsum_transformer = DataFrameMethodTransformer(
    columns = ['CRIM', 'INDUS', 'NOX'], 
    pd_method_name = 'cumsum',
    new_column_name = ['CRIM_duplicate', 'CRIM_INDUS', 'CRIM_INDUS_NOX'], 
    pd_method_kwargs = {'axis': 1}
)

In [14]:
boston_df_4 = cumsum_transformer.transform(boston_df)

In [15]:
boston_df_4[['CRIM', 'INDUS', 'NOX', 'CRIM_duplicate', 'CRIM_INDUS', 'CRIM_INDUS_NOX']].head()

,CRIM,INDUS,NOX,CRIM_duplicate,CRIM_INDUS,CRIM_INDUS_NOX
0,0.00632,2.31,0.538,0.00632,2.31632,2.85432
1,0.02731,7.07,0.469,0.02731,7.09731,7.56631
2,0.02729,7.07,0.469,0.02729,7.09729,7.56629
3,NaN,2.18,0.458,NaN,2.18000,2.63800
4,0.06905,2.18,0.458,0.06905,2.24905,2.70705


### Modulo

In [16]:
mod_transformer = DataFrameMethodTransformer(
    columns = ['INDUS'], 
    pd_method_name = 'mod',
    new_column_name = ['INDUS_mod_2'], 
    pd_method_kwargs = {'other': 2}
)

In [17]:
boston_df_5 = mod_transformer.transform(boston_df)

In [18]:
boston_df_5[['INDUS', 'INDUS_mod_2']].head()

,INDUS,INDUS_mod_2
0,2.31,0.31
1,7.07,1.07
2,7.07,1.07
3,2.18,0.18
4,2.18,0.18


### Less than

In [19]:
lt_transformer = DataFrameMethodTransformer(
    columns = ['INDUS'], 
    pd_method_name = 'lt',
    new_column_name = ['INDUS_lt_3'], 
    pd_method_kwargs = {'other': 3}
)

In [20]:
boston_df_6 = lt_transformer.transform(boston_df)

In [21]:
boston_df_6[['INDUS', 'INDUS_lt_3']].head()

,INDUS,INDUS_lt_3
0,2.31,True
1,7.07,False
2,7.07,False
3,2.18,True
4,2.18,True


### Absolute value

In [22]:
abs_transformer = DataFrameMethodTransformer(
    columns = ['INDUS'], 
    pd_method_name = 'abs',
    new_column_name = ['INDUS_abs']
)

In [23]:
boston_df_7 = abs_transformer.transform(boston_df)

In [24]:
boston_df_7[['INDUS', 'INDUS_abs']].head()

,INDUS,INDUS_abs
0,2.31,2.31
1,7.07,7.07
2,7.07,7.07
3,2.18,2.18
4,2.18,2.18


### Power

In [25]:
power_transformer = DataFrameMethodTransformer(
    columns = ['INDUS'], 
    pd_method_name = 'pow',
    new_column_name = ['INDUS_cubed'],
    pd_method_kwargs = {'other': 3}
)

In [26]:
boston_df_8 = power_transformer.transform(boston_df)

In [27]:
boston_df_8[['INDUS', 'INDUS_cubed']].head()

,INDUS,INDUS_cubed
0,2.31,12.326391
1,7.07,353.393243
2,7.07,353.393243
3,2.18,10.360232
4,2.18,10.360232


### Type setting

In [28]:
type_transformer = DataFrameMethodTransformer(
    columns = ['INDUS', 'CRIM'], 
    pd_method_name = 'astype',
    new_column_name = ['INDUS_str', 'CRIM_str'],
    pd_method_kwargs = {'dtype': 'str'}
)

In [29]:
boston_df_9 = type_transformer.transform(boston_df)

In [30]:
boston_df_9[['INDUS', 'CRIM', 'INDUS_str', 'CRIM_str']].dtypes

INDUS        float64
CRIM         float64
INDUS_str     object
CRIM_str      object
dtype: object